In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
!pip install yfinance -q

import yfinance as yf

In [8]:
#python должен быть версии 3.6-3.9 для установки numpy==1.19.5. Эта версия нужна для работы ta-lib
!pip uninstall -y numpy
!pip install numpy==1.24.4 -q

import numpy as np

Found existing installation: numpy 1.24.4
Uninstalling numpy-1.24.4:
  Successfully uninstalled numpy-1.24.4


In [4]:
# работа выполнялась на MacOS 10.15
# предварительно запустить brew install ta-lib
!pip install TA-Lib -q

import talib

In [5]:
!pip install plotly -q

import plotly.graph_objects as go

def linear_plot(df, title):
    fig = go.Figure([go.Scatter(x=df['date'], y=df['close'], mode='lines')])
    fig.update_layout(plot_bgcolor='white', 
                      xaxis_title='Date',
                      yaxis_title='Price',
                      title=title)
    fig.show()

def candlestick_plot(df, title):
    fig = go.Figure([go.Candlestick(x=df['date'], 
                                open=df['open'],
                                high=df['high'],
                                low=df['low'],
                                close=df['close'])])
    fig.update_layout(xaxis_rangeslider_visible=False,
                      plot_bgcolor='white',
                      xaxis_title='Date',
                      yaxis_title='Price',
                      title=title)
    fig.update_yaxes(fixedrange=False)
    fig.show()
    

In [6]:
import pandas as pd